<a href="https://colab.research.google.com/github/Denisganga/the_plant_doctor/blob/main/The_plant_doctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:

#importing the required modules
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
import torchvision.transforms as transforms

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
from torchvision import datasets
from torch.utils.data import random_split
import numpy as np
import random

In [86]:
#unzipping my dataset
import zipfile

zip_file_path =  "/content/drive/My Drive/the_plant_doctor/archive.zip"
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
  zip_ref.extractall(extract_path)

In [87]:
# Define the data directory (the path where your unzipped dataset is located)
data_dir="/content/dataset"

In [88]:
#setup device agnostic code(using the GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [89]:
#define the transformations
data_transforms =transforms.Compose([
    #make the model more robust to differently oriented images.
    transforms.RandomRotation(degrees=15),

    #random cropping of images to create multiple views of the same image
    transforms.RandomCrop(size=(224,224), padding=10),

    #Apply random color transformations to the images to make the model more invariant to changes in lighting and color
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2,hue=0.2),

    #help the model focus on important features and reduce noise
    transforms.GaussianBlur(kernel_size=3),

    #resize and crop the image to create variations in the field of view.
    transforms.RandomResizedCrop(size=(224,224), scale=(0.8, 1.0)),

    #introduce controlled occlusions or "erasing" of parts of the image during training
    transforms.RandomErasing(),

    #Randomly flip the image horizontally
    transforms.RandomHorizontalFlip(),

    #Convert the image to a PyTorch tensor
    transforms.ToTensor(),

     # Normalize the image based on typical RGB mean and standard deviation
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])



])

In [90]:
#create a dataset using ImageFolder
dataset= datasets.ImageFolder(data_dir, transform=data_transforms)

In [91]:
# Define class labels based on the dataset structure
classes = dataset.classes

In [92]:
#splitting the dataset into training and testing sets
train_size = int(0.8*len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset,[train_size, test_size])

In [93]:
print(len(dataset))
print(len(train_dataset))
print(len(test_dataset))

41276
33020
8256


In [94]:
#create a Dataloader for efficient dataloading and batch
batch_size = 7
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
num_samples = len(dataset)